# **Softmax Regression**

## **Libraries**

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import torch

## **Settings**

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### **Hyperparameters**

In [3]:
random_seed = 123
learning_rate = 0.1
num_epochs = 10
batch_size = 256

### **Architecture**

In [4]:
num_features = 784
num_classes = 10

## **MNIST Dataset**

In [6]:
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(),  download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor())

100%|██████████| 9.91M/9.91M [08:22<00:00, 19.7kB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 165kB/s]
100%|██████████| 1.65M/1.65M [00:05<00:00, 289kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 760kB/s]


In [7]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [8]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])


## **Model**

In [9]:
class SoftmaxRegression(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftmaxRegression, self).__init__()
        self.linear = torch.nn.Linear(num_features, num_classes)
        
        self.linear.weight.detach().zero_()
        self.linear.bias.detach().zero_()
        
    def forward(self, x):
        logits = self.linear(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

model = SoftmaxRegression(num_features=num_features, num_classes=num_classes)

model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [10]:
torch.manual_seed(random_seed)

def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    
    for features, targets in data_loader:
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
        
    return correct_pred.float() / num_examples * 100
    
for epoch in range(num_epochs):
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_dataset)//batch_size, cost))
            
    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))

Epoch: 001/010 | Batch 000/234 | Cost: 2.3026
Epoch: 001/010 | Batch 050/234 | Cost: 0.7693
Epoch: 001/010 | Batch 100/234 | Cost: 0.5321
Epoch: 001/010 | Batch 150/234 | Cost: 0.5617
Epoch: 001/010 | Batch 200/234 | Cost: 0.5501
Epoch: 001/010 training accuracy: 88.05%
Epoch: 002/010 | Batch 000/234 | Cost: 0.4793
Epoch: 002/010 | Batch 050/234 | Cost: 0.3686
Epoch: 002/010 | Batch 100/234 | Cost: 0.4535
Epoch: 002/010 | Batch 150/234 | Cost: 0.3731
Epoch: 002/010 | Batch 200/234 | Cost: 0.4791
Epoch: 002/010 training accuracy: 89.26%
Epoch: 003/010 | Batch 000/234 | Cost: 0.4057
Epoch: 003/010 | Batch 050/234 | Cost: 0.4456
Epoch: 003/010 | Batch 100/234 | Cost: 0.3770
Epoch: 003/010 | Batch 150/234 | Cost: 0.3893
Epoch: 003/010 | Batch 200/234 | Cost: 0.2996
Epoch: 003/010 training accuracy: 89.94%
Epoch: 004/010 | Batch 000/234 | Cost: 0.4298
Epoch: 004/010 | Batch 050/234 | Cost: 0.3918
Epoch: 004/010 | Batch 100/234 | Cost: 0.3756
Epoch: 004/010 | Batch 150/234 | Cost: 0.3902
Epo

In [ ]:
print('Test accuracy: %.2f%%' % (compute_accuracy(mmlxtendodel, test_loader)))

Test accuracy: 91.64%
